In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import drive
drive.mount('/content/drive')

def build_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

def run_experiment_with_predefined_data_nuclear_norm(df, n=2, m=2):
    mse_per_example = []
    rmse_per_example = []
    mae_per_example = []

    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_matrices', 'all_answers', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    unique_i_values = df['i'].unique()
    print("Unique 'i' values in dataframe:", unique_i_values)

    for i in unique_i_values:
        print(f"Processing for i={i} (number of prior examples)")
        experiments_for_i = df[df['i'] == i]

        mse_experiment = []
        mae_experiment = []
        rmse_experiment = []

        for _, row in experiments_for_i.iterrows():
            try:

                env = {"np": np, "array": np.array}
                matrices = eval(row['raw_questions'], env)

                answers = eval(row['raw_answers'], env)

                actual_nuclear_norm = float(answers[-1])

            except (SyntaxError, ValueError, NameError, TypeError) as e:
                print(f"Error parsing row {row['i']}-{row['j']}: {e}")
                continue

            X = np.array([matrix.flatten() for matrix in matrices])
            y = np.array(answers)  # Use all answers

            # Use all but the last matrix for training, and the last one for testing
            X_train, X_test = X[:-1], X[-1].reshape(1, -1)
            y_train, y_test = y[:-1], y[-1].reshape(1, -1)

            model = build_model((n * m,))
            model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

            y_pred = model.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))
            rmse = np.sqrt(mse)

            mse_experiment.append(mse)
            mae_experiment.append(mae)
            rmse_experiment.append(rmse)

            temp_df = pd.DataFrame({
                'i': [row['i']],
                'j': [row['j']],
                'input_matrices': [matrices],  # Store all matrices used in this experiment
                'all_answers': [answers],  # Store all answers (training and test)
                'actual_answers': [actual_nuclear_norm],  # The actual nuclear norm for the last matrix
                'predicted_answers': [y_pred.flatten()[0]],
                'mse': [mse],
                'rmse': [rmse],
                'mae': [mae]
            })
            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

        # Calculate average errors for each example size
        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.mean(rmse_experiment)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"Nuclear Norm - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    # Create summary DataFrame for average results
    df_average_results = pd.DataFrame({
        'i': unique_i_values,
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('LLM_results_Nuclear_Matrix_Norm_experiment_results.csv')
df_experiment_results, df_average_results = run_experiment_with_predefined_data_nuclear_norm(df, n=5, m=5)

df_experiment_results_path = '/content/drive/My Drive/2NN_df_experiment_results_nuclear_norm_2NN.csv'
df_average_results_path = '/content/drive/My Drive/2NN_df_average_results_nuclear_norm.csv'

df_experiment_results.to_csv(df_experiment_results_path, index=False)
df_average_results.to_csv(df_average_results_path, index=False)

print(f'Results saved to: {df_experiment_results_path} and {df_average_results_path}')

Unique 'i' values in dataframe: [  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100]
Processing for i=2 (number of prior examples)
1/1 [==============================] - 0s 75ms/step


<ipython-input-3-9ebd05bacf5e>:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)


1/1 [==============================] - 0s 51ms/step


1/1 [==============================] - 0s 53ms/step


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 52ms/step
Nuclear Norm - Examples: 4, MSE: 177030.08555359676, MAE: 409.91440761655684, RMSE: 409.91440761655684
Processing for i=5 (number of prior examples)
1/1 [==============================] - 0s 53ms/step
Nuclear Norm - Examples: 5, MSE: 139645.4431722277, MAE: 361.6044433381503, RMSE: 361.6044433381503
Processing for i=6 (number of prior examples)
1/1 [==============================] - 0s 50ms/step
Nuclear Norm - Examples: 6, MSE: 129592.46269087374, MAE: 349.9885941554883, RMSE: 349.9885941554883
Processing for i=7 (number of prior examples)
1/1 [==============================] - 0s 52ms/step
Nuclear Norm - Examples: 7, MSE: 118953.19633158478, MAE: 328.70356881135694, RMSE: 328.70356881135694
Processing for i=8 (number of prior examples)
1/1 [==============================] - 0s 51ms/step
Nuclear Norm - Examples: 8, MSE: 108353.01392276258, MAE: 314.70841964086407, RMSE: 314.7084196408

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

def run_sgd_experiment_with_predefined_data_nuclear_norm(df, n=2, m=2):

    mse_per_example = []
    rmse_per_example = []
    mae_per_example = []

    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_matrices', 'all_answers', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    unique_i_values = df['i'].unique()
    print("Unique 'i' values in dataframe:", unique_i_values)

    for i in unique_i_values:
        print(f"Processing for i={i} (number of prior examples)")
        experiments_for_i = df[df['i'] == i]

        mse_experiment = []
        mae_experiment = []
        rmse_experiment = []

        for _, row in experiments_for_i.iterrows():
            try:
                env = {"np": np, "array": np.array}
                matrices = eval(row['raw_questions'], env)

                answers = eval(row['raw_answers'], env)

                # Now, we need to take the last answer as the actual value
                actual_nuclear_norm = answers[-1]  # The last nuclear norm is the correct test value
                actual_nuclear_norm = float(actual_nuclear_norm)

            except (SyntaxError, ValueError, NameError, TypeError) as e:
                print(f"Error parsing row {row['i']}-{row['j']}: {e}")
                continue

            X = np.array([matrix.flatten() for matrix in matrices])
            y = np.array(answers)  # Use all the answers, and test on the last one

            # Use all but the last matrix for training, last for testing
            X_train, X_test = X[:-1], X[-1].reshape(1, -1)
            y_train, y_test = y[:-1], y[-1].reshape(1, -1)

            # Standardize the features
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            sgd_regressor = SGDRegressor(max_iter=1000, tol=1e-3, eta0=0.001, learning_rate='adaptive', alpha=0.1, warm_start=True)
            sgd_regressor.fit(X_train, y_train)

            y_pred = sgd_regressor.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))
            rmse = np.sqrt(mse)

            mse_experiment.append(mse)
            mae_experiment.append(mae)
            rmse_experiment.append(rmse)

            temp_df = pd.DataFrame({
                'i': [row['i']],
                'j': [row['j']],
                'input_matrices': [matrices],
                'all_answers': [answers],
                'actual_answers': [actual_nuclear_norm],
                'predicted_answers': [y_pred.flatten()[0]],
                'mse': [mse],
                'rmse': [rmse],
                'mae': [mae]
            })
            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.mean(rmse_experiment)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"SGD - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    df_average_results = pd.DataFrame({
        'i': unique_i_values,
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results


In [3]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('LLM_results_Nuclear_Matrix_Norm_experiment_results.csv')
df_experiment_results, df_average_results = run_sgd_experiment_with_predefined_data_nuclear_norm(df, n=5, m=5)

# Saving results to Google Drive
df_experiment_results_path = '/content/drive/My Drive/SGD_df_experiment_results_nuclear_norm_sgd_new.csv'
df_average_results_path = '/content/drive/My Drive/SGD_df_average_results_nuclear_norm_sgd_new.csv'

df_experiment_results.to_csv(df_experiment_results_path, index=False)
df_average_results.to_csv(df_average_results_path, index=False)

print(f'Results saved to: {df_experiment_results_path} and {df_average_results_path}')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np
import pandas as pd
import ast
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam

def build_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')  # Single output for nuclear norm
    ])
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

def run_experiment_with_predefined_data_cnn(df, n=2, m=2):

    mse_per_example = []
    rmse_per_example = []
    mae_per_example = []

    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_matrices', 'all_answers', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    unique_i_values = df['i'].unique()
    print("Unique 'i' values in dataframe:", unique_i_values)

    for i in unique_i_values:
        print(f"Processing for i={i} (number of prior examples)")
        experiments_for_i = df[df['i'] == i]

        mse_experiment = []
        mae_experiment = []
        rmse_experiment = []

        for j, row in experiments_for_i.iterrows():
            try:

                env = {"np": np, "array": np.array}
                matrices = eval(row['raw_questions'], env)
                answers = eval(row['raw_answers'], env)

                actual_nuclear_norm = float(answers[-1])

            except (SyntaxError, ValueError, NameError, TypeError) as e:
                print(f"Error parsing row {row['i']}-{row['j']}: {e}")
                continue

            X = np.array(matrices)
            y = np.array(answers)  # Use all nuclear norms

            # Use all but the last matrix for training, last for testing
            X_train, X_test = X[:-1], X[-1].reshape(1, n, m)
            y_train, y_test = y[:-1], np.array([actual_nuclear_norm]).reshape(1, -1)

            X_train = np.expand_dims(X_train, axis=-1)
            X_test = np.expand_dims(X_test, axis=-1)

            model = build_cnn_model((n, m, 1))
            model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

            y_pred = model.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))
            rmse = np.sqrt(mse)

            mse_experiment.append(mse)
            mae_experiment.append(mae)
            rmse_experiment.append(rmse)

            temp_df = pd.DataFrame({
                'i': [row['i']],
                'j': [row['j']],
                'input_matrices': [matrices],  # Store all matrices used in this experiment
                'all_answers': [answers],  # Store all answers (training and test)
                'actual_answers': [actual_nuclear_norm],  # The actual nuclear norm for the last matrix
                'predicted_answers': [y_pred.flatten()[0]],
                'mse': [mse],
                'rmse': [rmse],
                'mae': [mae]
            })
            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.mean(rmse_experiment)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"CNN - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    df_average_results = pd.DataFrame({
        'i': unique_i_values,
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results


In [ ]:
df = pd.read_csv('LLM_results_Nuclear_Matrix_Norm_experiment_results.csv')
df_experiment_results, df_average_results = run_experiment_with_predefined_data_cnn(df, n=5, m=5)

# Saving results to Google Drive
df_experiment_results_path = '/content/drive/My Drive/CNN_df_experiment_results_nuclear_norm_sgd.csv'
df_average_results_path = '/content/drive/My Drive/CNN_df_average_results_nuclear_norm_sgd.csv'

df_experiment_results.to_csv(df_experiment_results_path, index=False)
df_average_results.to_csv(df_average_results_path, index=False)

print(f'Results saved to: {df_experiment_results_path} and {df_average_results_path}')

Unique 'i' values in dataframe: [  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100]
Processing for i=2 (number of prior examples)
1/1 [==============================] - 0s 100ms/step


<ipython-input-7-378af799a52c>:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 62ms/step
CNN - Examples: 4, MSE: 66808.31261922032, MAE: 239.54897853012568, RMSE: 239.54897853012568
Processing for i=5 (number of prior examples)
1/1 [==============================] - 0s 61ms/step
CNN - Examples: 5, MSE: 55317.48605980517, MAE: 213.05109170827123, RMSE: 213.05109170827123
Processing for i=6 (number of prior examples)
1/1 [==============================] - 0s 67ms/step
CNN - Examples: 6, MSE: 51793.9421608872, MAE: 203.51576604653945, RMSE: 203.51576604653945
Processing for i=7 (number of prior examples)
1/1 [==============================] - 0s 65ms/step
CNN - Examples: 7, MSE: 63953.4515040728, MAE: 227.39439091174674, RMSE: 227.39439091174674
Processing for i=8 (number of prior examples)
1/1 [==============================] - 0s 62ms/step
CNN - Examples: 8, MSE: 52246.3498239784, MAE: 198.74165113235736, RMSE: 198.74165113235736
Processing for i=9 (number of prior example